## Лабораторная работа #6: Introduction to neural nets 

В данной лабораторной работе вам предстоит познакомиться с искуственными нейронными сетями. Вообще нейронные сети применимы к широкому спектру задач, однако в области NLP они, как правило, решают задачи классификации. Поэтому сегодня мы будем обучать нейронные сети именно этой задаче.

In [1]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import random
import torch.nn as nn
import numpy as np
import torch
from torch.autograd import Variable
from sklearn.datasets import load_wine, load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

Рассмотрим самый базовый датасет для классифицирующих моделей -- $\href{https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html}{iris}$. Каждая запись в этом датасете предсталяет из себя набор численных характеристик некоторого цветка ириса, целевым параметром является лейбл класса. 

In [4]:
X, y = load_iris(return_X_y=True)
n_categories = max(y) + 1
print(" Число цветков (семлов) = {}\n Число численных характеристик (фичей) цветка = {}\n Количество классов = {}\n".format(X.shape[0], X.shape[1], n_categories))
print(" Вектора фичей выглядят так:\n\n {}".format(X[::30]))
print(" А целевые значения, лейблы классов, выглядят так:\n {}".format(y[::30]))

 Число цветков (семлов) = 150
 Число численных характеристик (фичей) цветка = 4
 Количество классов = 3

 Вектора фичей выглядят так:

 [[5.1 3.5 1.4 0.2]
 [4.8 3.1 1.6 0.2]
 [5.  2.  3.5 1. ]
 [5.5 2.6 4.4 1.2]
 [6.9 3.2 5.7 2.3]]
 А целевые значения, лейблы классов, выглядят так:
 [0 0 1 1 2]


Задача классификации состоит в предсказании лейблов класса по данным векторам фичей. Эту задачу можно решать классифицирующими нейронными сетями. Чем мы и займёмся!

### Подготовка данных

Прежде чем мы приступим к написанию нейронной сети, давайте подготовим данные.

Во-первых, хорошей практикой является нормализация данных. Нейронные сети лучше обучаются, если все численные характеристики в векторах численных характеристик располагаются в отрезке [0, 1]. Это можно сделать с помощью MinMaxScaler, который для каждой фичи находит её максимальное и минимальное значение среди всех семплов и относительно них нормализует соответствующий компонент каждого вектора фич.

Во-вторых, категориальные характеристики. Целевым значением является лейбл класса, который может принимать только три значения: {0, 1, 2}. Если мы будем хранить это значение как число, то предопредилим нежелательные отношения между классами -- так, класс $1$ будет рассматриваться моделью как более близкий к классу $2$, чем класс $0$, что вовсе не детерминированно условиями задачи или входными данными. Нужно получить такое представление классов, чтоб расстояние между предсталениями двух любых различных классов было одинаково. Это можно сделать с помощью техники $\href{https://en.wikipedia.org/wiki/One-hot}{One-Hot Encoding}$: давайте кодировать категориальные характеристики с помощью векторов размерности $n = $число различных классов. Каждый такой вектор будет содержать нули во всех своих компонентах кроме той, которая соответствует номеру класса, который мы пытаемся кодировать: в этой компоненте будет стоять единица. Например, последовательность [1, 0, 2, 0...] будет закодирована как [[0, 1, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0...]]

In [5]:
X, y = load_iris(return_X_y=True)
scaler = MinMaxScaler()
X = scaler.fit_transform(X, y)
y = np.array([np.array([0.0 if i != cat else 1.0 for i in range(n_categories)]) for cat in y])

print(" Теперь вектора фичей выглядят так:\n\n {}".format(X[::30]))
print(" А целевые значения, лейблы классов, выглядят так:\n {}".format(y[::30]))

 Теперь вектора фичей выглядят так:

 [[0.22222222 0.625      0.06779661 0.04166667]
 [0.13888889 0.45833333 0.10169492 0.04166667]
 [0.19444444 0.         0.42372881 0.375     ]
 [0.33333333 0.25       0.57627119 0.45833333]
 [0.72222222 0.5        0.79661017 0.91666667]]
 А целевые значения, лейблы классов, выглядят так:
 [[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


Теперь разделим данные на тренировочные и тестовые. Для нейронных сетей мы будем использовать библиотеку $\href{https://pytorch.org/}{PyTorch}$, обладающую своими типами и строгой типизацией, так что нам нужно привести данные к её типам.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

X_train = Variable(torch.from_numpy(X_train))
X_test = Variable(torch.from_numpy(X_test))
y_train = Variable(torch.from_numpy(y_train))
y_test = Variable(torch.from_numpy(y_test))

Давайте придумаем архитектуру классифицирующей нейронной сети.

Число нейронов во входном слое должно соответствовать размерности фича-веткора, в нашем случае это 4. Число нейронов в выходном слое должно соответствовать числу классов, в нашем случае это 3.

Очень часто в задачах классификации в качестве функции активации для последнего слоя нейронов используется функция $\href{https://ru.wikipedia.org/wiki/Softmax}{SoftMax}$. Эта функция преобразовывает вектор таким образом, что каждая его компонента становится неотрицательной, а сумма компонент равняется 1. Из-за этого компоненту под номером i получающегося вектора можно рассматривать как степень уверенности нашей сети в том, что обрабатываемый семпл принадлежит к классу i. Именно таким образом (one-hot) закодированы правильные ответы.

Опишем нейронную сеть с одним скрытым слоем при помощи библиотеки $\href{https://pytorch.org/}{PyTorch}$. Это актуальный и очень серёьзный, но при этом гибкий и удобный инструмент. Смотрите, как просто и интуитивно-понятно!

In [7]:
i_size = X.shape[1]     #число нейронов входного слоя
h_size = 10             #число нейронов скрытого слоя
o_size = n_categories   #число нейронов выходного слоя

intuitive_model = nn.Sequential(
    nn.Linear(i_size, h_size), #линейная операция сложения, числа соответствуют размерностям входного и выходного векторов
    nn.ReLU(),                 #функция активации RELU
    nn.Linear(h_size, o_size),
    nn.Softmax()               #функция активации SoftMax
).double()                     #строгая типизация строга

Теперь опишем функцию train_batch, которая принимает пачку примеров, вычисляет усреднённую ошибку нейронной сети на них и соответствующим образом меняет её веса. Это возможно благодаря методу обратного распространения ошибки (gradient descent), который, конечно, помните из лекции. В ходе обратного распространения ошибки необходимо считать градиенты от функции ошибки по векторам весов сети.

Хорошие новости: нам не придётся вручную считать градиенты, потому что torch может делать это сам. Нам нужно лишь определить оптимайзер (возьмем стохастический градиентный спуск), функцию ошибки (возьмем бинарную кросс-энтропию) и скорость обучения (learning rate, возьмем 0.05).

Когда эти параметры определены, можно вычислять градиенты и менять веса при помощи простых команд.

In [8]:
loss = nn.BCELoss() #функция ошибки

opt= torch.optim.SGD(intuitive_model.parameters(),lr=0.05) #оптимайзер и скорость обучения


def train_batch(model, X, y):  # функция, обучающая модель на пачке примеров
    y_pred = model(X)          # получим ответ сети для каждого из примеров
    ls = loss(y_pred, y)       # вычислим функцию ошибки
    opt.zero_grad()            
    ls.backward()              # посчитаем градиенты
    opt.step()                 # изменяем веса сети
    return ls

Реализуем функцию train_model, обучающую нейронную сеть в течение n_epochs эпох. Одна эпоха соответствует одному полному просмотру тренировочного датасета. Как уже было отмечено, мы будем обучать сеть пачками (batches) примеров. В рамках одной эпохи мы будем перемешивать тренировочную выборку, делить её на пачки по batch_size примеров, и "скармливать" их модели.

Давайте обучим нашу сеть на 20 эпохах и оценим результаты её работы на тестовой выборке.

In [9]:
batch_size = 4 #рамзер пачки

def train_model(model, n_epochs, print_every):
    last_test_loss = None
    for epoch in range(n_epochs):
        n = X_train.shape[0]
        random_permutation = torch.randperm(n) #возьмем случайную перестановку примеров
        total_loss = 0
        
        for i in range(0,n,batch_size):
            indices = random_permutation[i:i+batch_size] #индексы примеров очередной пачки
            batch_x, batch_y = X_train[indices], y_train[indices] # "скормим" их модели
            total_loss += train_batch(model, batch_x, batch_y) #посчитаем суммарную ошибку на трейне
                
        if(print_every > 0 and (epoch + 1) % print_every == 0):
            pred = intuitive_model(X_test)               #посчитает результат и ошибку на тесте
            last_test_loss = loss(pred, y_test)
            print("Epoch #{}\n".format(epoch + 1))
            print("Error on train = {}\n".format(total_loss / (n // batch_size)))
            print("Error on test = {}\n".format(loss(pred, y_test)))
            
    return last_test_loss
    
ll = train_model(intuitive_model, 20, 2)

/Users/vladiknaska/Documents/se/nlp/nlp_venv/lib/python3.6/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch #2

Error on train = 0.62274412428104

Error on test = 0.6770202044165307

Epoch #4

Error on train = 0.5956905052660938

Error on test = 0.6773206542227583

Epoch #6

Error on train = 0.5632591504464858

Error on test = 0.6628877437698222

Epoch #8

Error on train = 0.5259868797628399

Error on test = 0.6428719566765821

Epoch #10

Error on train = 0.48684064254919235

Error on test = 0.6211214516960782

Epoch #12

Error on train = 0.44918573967844005

Error on test = 0.5864666373491116

Epoch #14

Error on train = 0.41390112730001805

Error on test = 0.5452615465240553

Epoch #16

Error on train = 0.3827378800040876

Error on test = 0.5146768992346182

Epoch #18

Error on train = 0.35626390928306767

Error on test = 0.4802745427015656

Epoch #20

Error on train = 0.33237665846653835

Error on test = 0.4532043177655273



А теперь давайте оценим нашу модель на тестовом сете -- выведем confusion matrix и число неверно классифицированных семплов.

Confusion matrix -- матрица размерности n_classes x n_classes, число в i-й строке и j-м столбце соответствует количеству примеров из класса i, которые были классифицированы как j. Сумма главной диагонали, таким образом, соответствует количеству правильных ответов. 

In [13]:
def evaluate_model(model, print_matrix = True):
    with torch.no_grad():
        y_pred = model(X_test) #делаем предсказание на тесте
        y_pred_new = torch.argmax(y_pred, dim=1) #ответ сети -- номер компоненты вектора с наибольшим значением. Этот номер соответстует классу.
        y_test_true = torch.argmax(y_test, dim=1) #обратно кодируем правильные ответы, чтоб узнать правильные лейблы.
        if(print_matrix):
            print(confusion_matrix(y_test_true, y_pred_new))
        missclass = sum([int(pred != true) for (pred, true) in zip(y_pred_new, y_test_true)]) #вот так можно посчитать число неправильных ответов
    return missclass

print("Число неправильно классифицированных примеров = {}".format(evaluate_model(intuitive_model)))

[[19  0  0]
 [ 1  0 23]
 [ 0  0  7]]
Число неправильно классифицированных примеров = 24


Что ж, недурно! Но, может быть, можно лучше? Может быть можно выбрать такие параметры сети, что ни один пример не будет классифицирован неправильно?

Есть ещё один способ задания сети с помощью PyTorch: кастомизация нейросети путём наследования от базового класса torch.nn.module. Наследующий класс должен реализовать функцию-констукртор __init__, инициализирущую веса сети, и функцию forward, совершающую прямое прохождение по сети. В такой имплементации любые параметры сети (например, количество нейронов в скрытом слое или функция активации на нём) можно сделать параметрами её конструктора.

Давайте реализуем класс CustomModel, соответствующий нейронной сети с одним скрытым слоем, число нейронов и функция активации которого будет передаваться на стадии инициализации. Функции train_batch и train_model можно сделать методами нового класса, а все вспомогательные объекты (оптимизатор, learning rate, batch_size) -- его параметрами.

In [14]:
class CustomModel(torch.nn.Module):
    def __init__(self, hidden_size=8, activation=torch.nn.ReLU()): #конструктор сети принимает на вход число нейронов скрытого слоя и активирующую его функцию
        super(CustomModel,self).__init__()
        self.l1 = torch.nn.Linear(i_size, hidden_size)   
        self.act=activation                              #конкретная функция activation будет получена как параметр
        self.l2 = torch.nn.Linear(hidden_size, o_size)
        self.softmax = torch.nn.Softmax()
        self.opt = torch.optim.Adam(self.parameters(), lr=0.05)
        self.batch_size = 4
        
    """NB: Во всём остальном -- то же самое, что и intuitive_model"""
    
    def forward(self,x):
        h=self.act(self.l1(x))
        return self.softmax(self.l2(h))

    def train_batch(self, X, y):
        y_pred = self(X)
        ls = loss(y_pred, y)
        self.opt.zero_grad()
        ls.backward()
        self.opt.step()
        return ls
    
    def train_model(self, n_epochs, print_every):
        last_test_loss = None
        for epoch in range(n_epochs):
            n = X_train.shape[0]
            random_permutation = torch.randperm(n)
            total_loss = 0
            for i in range(0,n,self.batch_size):
                indices = random_permutation[i:i+self.batch_size]
                batch_x, batch_y = X_train[indices], y_train[indices]
                total_loss += self.train_batch(batch_x, batch_y)

Давайте найдем лучшие значения hidden_size и activation, а также оптимальное число обучающих эпох для нашей задачи. Есть множество техник поиска гиперпараметров, самый простой из них -- выбрать несколько значений для разных параметров, перебрать все комбинации и найти лучшее сочетание. Давайте это и сделаем с помощью функции evaluate_model.

In [15]:
import itertools

activations = [torch.nn.ReLU(), torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.SELU(), torch.nn.Hardtanh()] #перебираемые функции активации
hidden_sizes = [3, 4, 7, 10, 20] #перебираемое число нейронов скрытого слоя
epochs = [25, 50] #перебираемое число эпох
best_misses = len(y_test) #лучший результат в смысле количества неправильно классифицированных семплов
best_paramset = None

for paramset in (itertools.product(epochs, activations, hidden_sizes)): #paramset -- tuple, содержащий значения каждого из параметров
    e, a, h = paramset
    model = CustomModel(hidden_size = h, activation = a).double() #создаём нейронную сеть с перечисленными параметрами
    model.train_model(e, 0) #тренируем её в течение %e% эпох 
    misses = evaluate_model(model, print_matrix = False) #количеств неправильно классифицируемых примеров
    if(best_misses > misses): #если результат лучше -- обновляем
        best_misses = misses
        best_paramset = paramset
        
print("Best paramset: n_epochs={}, activation_function ={}, hidden_size = {}\n. Micclassifications = {}.\n".format(best_paramset[0],best_paramset[1],best_paramset[2],best_misses))


/Users/vladiknaska/Documents/se/nlp/nlp_venv/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Best paramset: n_epochs=25, activation_function =ReLU(), hidden_size = 7
. Micclassifications = 3.



## Задание: а теперь ваша очередь

Ваше домашнее задание состоит в том, чтоб по аналогии построить классифицирующую модель для датасета wine. Вам необходимо построить модель, предсказывающую класс вина по его численным характеристикам. Если точнее, от вас требуется:

1. Подготовить данные: произвести нормализацию численных данных и one-hot векторизацию категориальных, разделить выборку на тренировочную и тестовую
2. Построить и обучить классифицирующую модель на основе нейронной сети, содержащую один скрытый слой. Возьмите такое количество эпох обучения, которое достататочно для того, чтоб функция ошибки перестала падать. Все остальные параметры сети -- на ваше усмотрение.
3. Выберите несколько значений параметров hidden_size (число нейронов скрытого слоя), batch_size (число примеров в пачке) и learning_rate(скорость обучения). Опираясь на функцию evaluate_model, найдите лучшую комбинацию параметров. Выведите confusion matrix на тесте для нейронной сети с оптимальными параметрами.

In [2]:
X, y = load_wine(return_X_y=True)
n_categories = max(y) + 1
print(" Число проб вина (семплов) = {}\n Число численных характеристик (фичей) каждой пробы вина = {}\n Количество классов = {}\n".format(X.shape[0], X.shape[1], n_categories))
print(" Вектора фичей выглядят так:\n\n {}".format(X[::30]))
print(" А целевые значения, лейблы классов, выглядят так:\n {}".format(y[::30]))

 Число проб вина (семплов) = 178
 Число численных характеристик (фичей) каждой пробы вина = 13
 Количество классов = 3

 Вектора фичей выглядят так:

 [[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
  2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
 [1.373e+01 1.500e+00 2.700e+00 2.250e+01 1.010e+02 3.000e+00 3.250e+00
  2.900e-01 2.380e+00 5.700e+00 1.190e+00 2.710e+00 1.285e+03]
 [1.233e+01 1.100e+00 2.280e+00 1.600e+01 1.010e+02 2.050e+00 1.090e+00
  6.300e-01 4.100e-01 3.270e+00 1.250e+00 1.670e+00 6.800e+02]
 [1.208e+01 1.830e+00 2.320e+00 1.850e+01 8.100e+01 1.600e+00 1.500e+00
  5.200e-01 1.640e+00 2.400e+00 1.080e+00 2.270e+00 4.800e+02]
 [1.145e+01 2.400e+00 2.420e+00 2.000e+01 9.600e+01 2.900e+00 2.790e+00
  3.200e-01 1.830e+00 3.250e+00 8.000e-01 3.390e+00 6.250e+02]
 [1.350e+01 3.120e+00 2.620e+00 2.400e+01 1.230e+02 1.400e+00 1.570e+00
  2.200e-01 1.250e+00 8.600e+00 5.900e-01 1.300e+00 5.000e+02]]
 А целевые значения, лейблы классов, в

Подготовка данных:

In [3]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X, y)
y = np.array([np.array([0.0 if i != cat else 1.0 for i in range(n_categories)]) for cat in y])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

X_train = Variable(torch.from_numpy(X_train))
X_test = Variable(torch.from_numpy(X_test))
y_train = Variable(torch.from_numpy(y_train))
y_test = Variable(torch.from_numpy(y_test))

In [5]:
i_size = X.shape[1]     #число нейронов входного слоя
h_size = 10             #число нейронов скрытого слоя
o_size = n_categories   #число нейронов выходного слоя

intuitive_model = nn.Sequential(
    nn.Linear(i_size, h_size), #линейная операция сложения, числа соответствуют размерностям входного и выходного векторов
    nn.ReLU(),                 #функция активации RELU
    nn.Linear(h_size, o_size),
    nn.Softmax()               #функция активации SoftMax
).double()                     #строгая типизация строга

In [10]:
def train_batch(model, X, y):  # функция, обучающая модель на пачке примеров
    y_pred = model(X)          # получим ответ сети для каждого из примеров
    ls = loss(y_pred, y)       # вычислим функцию ошибки
    opt.zero_grad()            
    ls.backward()              # посчитаем градиенты
    opt.step()                 # изменяем веса сети
    return ls

def train_model(model, n_epochs, print_every):
    last_test_loss = None
    for epoch in range(n_epochs):
        n = X_train.shape[0]
        random_permutation = torch.randperm(n) #возьмем случайную перестановку примеров
        total_loss = 0
        
        for i in range(0,n,batch_size):
            indices = random_permutation[i:i+batch_size] #индексы примеров очередной пачки
            batch_x, batch_y = X_train[indices], y_train[indices] # "скормим" их модели
            total_loss += train_batch(model, batch_x, batch_y) #посчитаем суммарную ошибку на трейне
                
        if(print_every > 0 and (epoch + 1) % print_every == 0):
            pred = intuitive_model(X_test)               #посчитает результат и ошибку на тесте
            last_test_loss = loss(pred, y_test)
            print("Epoch #{}\n".format(epoch + 1))
            print("Error on train = {}\n".format(total_loss / (n // batch_size)))
            print("Error on test = {}\n".format(loss(pred, y_test)))
            
    return last_test_loss

def evaluate_model(model, print_matrix = True):
    with torch.no_grad():
        y_pred = model(X_test) #делаем предсказание на тесте
        y_pred_new = torch.argmax(y_pred, dim=1) #ответ сети -- номер компоненты вектора с наибольшим значением. Этот номер соответстует классу.
        y_test_true = torch.argmax(y_test, dim=1) #обратно кодируем правильные ответы, чтоб узнать правильные лейблы.
        if(print_matrix):
            print(confusion_matrix(y_test_true, y_pred_new))
        missclass = sum([int(pred != true) for (pred, true) in zip(y_pred_new, y_test_true)]) #вот так можно посчитать число неправильных ответов
    return missclass

In [11]:
batch_size = 4

loss = nn.BCELoss() #функция ошибки

opt= torch.optim.SGD(intuitive_model.parameters(),lr=0.05) #оптимайзер и скорость обучения

ll = train_model(intuitive_model, 20, 2)

Epoch #2

Error on train = 0.20721330115755246

Error on test = 0.18303200645044995

Epoch #4

Error on train = 0.179096156661393

Error on test = 0.1570367959137998

Epoch #6

Error on train = 0.15794005832017358

Error on test = 0.13917329383180435

Epoch #8

Error on train = 0.14215861094005677

Error on test = 0.12426008252310923

Epoch #10

Error on train = 0.128143775487533

Error on test = 0.1107270484303572

Epoch #12

Error on train = 0.11499175289291946

Error on test = 0.09935678245067647

Epoch #14

Error on train = 0.1040699617229757

Error on test = 0.0943608661613263

Epoch #16

Error on train = 0.09666841192501965

Error on test = 0.08458354507216463

Epoch #18

Error on train = 0.09043513987942763

Error on test = 0.07868531671303282

Epoch #20

Error on train = 0.08430274194807134

Error on test = 0.0751684090489179



In [12]:
print("Число неправильно классифицированных примеров = {}".format(evaluate_model(intuitive_model)))

[[20  0  0]
 [ 1 23  1]
 [ 0  0 14]]
Число неправильно классифицированных примеров = 2


Результаты итак очень хорошие, но эксперимента ради попробую унаследоваться от  torch.nn.Model

In [13]:
class CustomModel(torch.nn.Module):
    def __init__(self, hidden_size=8, activation=torch.nn.ReLU()): #конструктор сети принимает на вход число нейронов скрытого слоя и активирующую его функцию
        super(CustomModel,self).__init__()
        self.l1 = torch.nn.Linear(i_size, hidden_size)   
        self.act=activation                              #конкретная функция activation будет получена как параметр
        self.l2 = torch.nn.Linear(hidden_size, o_size)
        self.softmax = torch.nn.Softmax()
        self.opt = torch.optim.Adam(self.parameters(), lr=0.05)
        self.batch_size = 4
        
    """NB: Во всём остальном -- то же самое, что и intuitive_model"""
    
    def forward(self,x):
        h=self.act(self.l1(x))
        return self.softmax(self.l2(h))

    def train_batch(self, X, y):
        y_pred = self(X)
        ls = loss(y_pred, y)
        self.opt.zero_grad()
        ls.backward()
        self.opt.step()
        return ls
    
    def train_model(self, n_epochs, print_every):
        last_test_loss = None
        for epoch in range(n_epochs):
            n = X_train.shape[0]
            random_permutation = torch.randperm(n)
            total_loss = 0
            for i in range(0,n,self.batch_size):
                indices = random_permutation[i:i+self.batch_size]
                batch_x, batch_y = X_train[indices], y_train[indices]
                total_loss += self.train_batch(batch_x, batch_y)

In [16]:
import itertools

activations = [torch.nn.ReLU(), torch.nn.Tanh(),
               torch.nn.Sigmoid(), torch.nn.SELU(),
               torch.nn.Hardtanh()] #перебираемые функции активации
hidden_sizes = [3, 4, 7, 10, 20] #перебираемое число нейронов скрытого слоя
epochs = [1, 3, 5, 10, 25, 50] #перебираемое число эпох
best_misses = len(y_test) #лучший результат в смысле количества неправильно классифицированных семплов
best_paramset = None

for paramset in (itertools.product(epochs, activations, hidden_sizes)): #paramset -- tuple, содержащий значения каждого из параметров
    e, a, h = paramset
    model = CustomModel(hidden_size = h, activation = a).double() #создаём нейронную сеть с перечисленными параметрами
    model.train_model(e, 0) #тренируем её в течение %e% эпох 
    misses = evaluate_model(model, print_matrix = False) #количеств неправильно классифицируемых примеров
    if(best_misses > misses): #если результат лучше -- обновляем
        best_misses = misses
        best_paramset = paramset
        
print("Best paramset: n_epochs={}, activation_function ={}, hidden_size = {}\n. Micclassifications = {}.\n".format(best_paramset[0],best_paramset[1],best_paramset[2],best_misses))



/Users/vladiknaska/Documents/se/nlp/nlp_venv/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Best paramset: n_epochs=3, activation_function =Hardtanh(min_val=-1.0, max_val=1.0), hidden_size = 3
. Micclassifications = 0.

